<a href="https://colab.research.google.com/github/harshaelon/Financial_Querying_Langchain/blob/main/Financial_Querieying_using_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PDF Query Using Langchain

In [5]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.2 MB/s eta 0:00:00


In [3]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.8 MB/s eta 0:00:00


In [6]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [18]:
import os
os.environ["OPENAI_API_KEY"] = ""
#os.environ["SERPAPI_API_KEY"] = ""

In [8]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('/content/financialguide.pdf')

In [9]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [10]:
raw_text

"Copyright Plain English Campaign.\nThe A to Z of financial terms\nThis guide is not intended to be the final word. If you have any suggestions, corrections or\nimprovements, please do contact us. You can contact us at:\nPO Box 3\nNew Mills\nHigh Peak\nSK22 4QP\n01663 744409 (phone)\n01663 747038 (fax)\ninfo@plainenglish.co.uk\nwww.plainenglish.co.uk\nCopyright\nPlain English Campaign owns the copyright on this guide. You may save one copy of the guide to\ndisk, and print one copy out for your personal use. You may not make more than one copy without\nour permission.\nIntroduction\nPlain English Campaign brings light to technical language which is dark and troubling for many\npeople. This excellent guide will help ordinary people understand financial, tax and legal\ndocuments. It will also help and encourage those who write such documents to use language which\nthe intended audience can understand. The guide shows that there is no need for technical\nlanguage to baffle and befuddle.\nW

In [11]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [12]:
len(texts)

122

In [19]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [20]:
document_search = FAISS.from_texts(texts, embeddings)

In [21]:
document_search


In [22]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [23]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [24]:
query = "what is the document about?"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


" The document is about a guide created by Plain English Campaign to help people understand financial, tax, and legal documents and encourage the use of language that is easy to understand. It also mentions the Campaign's impact on improving communication between government, business, and the public."

In [25]:
query = "What are some important terms in finance industry??"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Some important terms in the finance industry include interest-rate swap, interim dividend, intestacy, investment grade issuers, investment trust, ISA, issued share capital, financial statement, financial year, fiscal, fixed asset, fixed interest rate, floating charge, floating exchange rates, trade creditors, trade debtors, trading account, trading profit, transferable securities, transferee, facility, final dividend, and finance lease.'

In [26]:
query = "can you tell me about taxation policy? and its need??"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Taxation policy refers to the principles and guidelines that a government follows in order to collect taxes from its citizens and businesses. These policies are designed to generate revenue for the government, fund public services and programs, and influence economic behavior.\n\nThe need for taxation policy arises from the fact that governments need a steady stream of income to fund their operations and provide essential services to citizens. This includes things like education, healthcare, infrastructure, and national defense. Without sufficient tax revenue, a government may struggle to meet these obligations and maintain a stable economy.\n\nTaxation policy also serves as a tool for government to influence economic behavior. By adjusting tax rates and implementing incentives or penalties, governments can encourage or discourage certain activities, such as investment, consumption, and saving. This can have a significant impact on the overall health and growth of the economy.\n\nOve

In [27]:
query = "so what are key market indicators? what is this PE ratio, debt-equity ration and all??"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

" Key market indicators are financial measures used to track the performance of a specific market or industry. These indicators can include things like stock indices, interest rates, and economic data. The PE ratio, or price-to-earnings ratio, is a measure of a company's stock price relative to its earnings. The debt-equity ratio is a measure of a company's financial leverage, calculated by dividing its total debt by its total equity. These indicators can be used to evaluate the health and potential of a market or company."

In [28]:
query = "What is Bull Market??"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' A bull market is a financial market in which prices are rising or are expected to rise. This often leads to investors buying shares in the hopes of selling them at a profit in the future.'